In [ ]:
import torch
import matplotlib.pyplot as plt
from pytorch3d.io import load_objs_as_meshes
from pathlib import Path
from neural_poisson.data.prepare import (
    extract_surface_data,
    uniform_sphere_cameras,
    sample_empty_space_points,
    subsample_points,
    estimate_vector_field_nearest_neighbors
)

# settings
image_size = 512
segments = 10
device = "cuda"
model_id = "1a04e3eab45ca15dd86060f189eb133"

# define the paths
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCoreTiny/02691156"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

# load the mesh and the cameras
mesh = load_objs_as_meshes([shapenet_path], device=device)
cameras = uniform_sphere_cameras(segments=segments, device=device)

# visualize a camera
elev = 8
azim = 1
data = extract_surface_data(
    camera=cameras[elev + azim * segments],
    mesh=mesh,
    image_size=image_size,
)
normal = torch.clip(data["normal_map"][0], 0.0, 1.0)
plt.imshow(normal.detach().cpu().numpy())

In [ ]:
normals = []
points = []
for camera in cameras[:5]:
    data = extract_surface_data(camera=camera, mesh=mesh, image_size=image_size)
    normals.append(data["normals"])
    points.append(data["points"])
normals = torch.cat(normals) 
points = torch.cat(points) 
vectors = estimate_vector_field_nearest_neighbors(
    points=points,
    normals=normals,
    query=points,
    k=10,
    sigma=1.0,
)

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points.detach().cpu().numpy())

pcd.normals = o3d.utility.Vector3dVector(vectors.detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
camera = cameras[0]
data = extract_surface_data(camera=camera, mesh=mesh, image_size=image_size)
p_e = sample_empty_space_points(
    points=data["points"],
    camera=camera,
    surface_threshold=1.0,
    samples=4,
)
p_e = subsample_points(points=p_e, resolution=0.01)


In [ ]:
import open3d as o3d

k = 20
sigma = 1.0
threshold = 30 
points = data["points"]  # the surface points
normals = data["normals"]  # the normals on the surface
query = data["points"]  # the target points to evaluate the vector field
chunk_size = 1_000
normalize = True

vectors = []
for q in torch.split(query, chunk_size):
    # compute the distances
    distances = torch.cdist(q, points, p=2)
    distances, indices = torch.topk(distances, k, dim=1, largest=False)

    # gaussian-weighted average of the k nearest neighbors
    weights = torch.exp(-distances / (2 * sigma))

    # compute the clusters
    cluster_idxs = torch.full_like(indices, -1)
    cluster_idxs[:, 0] = 0
    for i in range(1, k):
        prev_idxs = cluster_idxs[:, :i]
        prev_max_cluster = prev_idxs.max(dim=-1).values

        # default is just the next index
        current_normal = normals[indices][:, i, :]
        current_idxs = prev_max_cluster + 1
        tmp_cluster_similarity = -torch.ones(q.shape[0]).to(q) # fill with -1

        # compute the previous cluster vectors
        for j in range(0, i):
            # activate the current cluster by disabeling all others
            cluster_weights = weights.clone()
            cluster_weights[cluster_idxs != j] = 0.0
            cluster_vector = (normals[indices] * cluster_weights.unsqueeze(-1)).sum(-2)
            cluster_vector /= cluster_weights.sum(-1, keepdim=True)

            # compute cosine similartiy between cluster vector and current vector
            similarity = (cluster_vector * current_normal).sum(-1)
            similarity /= torch.linalg.vector_norm(cluster_vector, dim=-1)
            similarity /= torch.linalg.vector_norm(current_normal, dim=-1)

            # compute the angle 
            theta = torch.rad2deg(torch.acos(similarity))

            # update the best cluster
            mask = (similarity > tmp_cluster_similarity) & (theta <= threshold) 
            tmp_cluster_similarity[mask] = similarity[mask]
            current_idxs[mask] = j

        # if there is a cluster that matches 
        cluster_idxs[:, i] = current_idxs
    
    # evalute the cluster normals and centers
    cluster_vectors = []
    cluster_centers = []
    for j in range(0, k):
        # activate the current cluster by disabeling all others
        cluster_weights = weights.clone()
        cluster_weights[cluster_idxs != j] = 0.0
        cluster_vector = (normals[indices] * cluster_weights.unsqueeze(-1)).sum(-2)
        cluster_vector /= cluster_weights.sum(-1, keepdim=True)
        cluster_vectors.append(cluster_vector)
        # activate the current cluster centers
        cluster_center = (points[indices] * cluster_weights.unsqueeze(-1)).sum(-2)
        cluster_centers.append(cluster_center)
    cluster_centers = torch.stack(cluster_centers, dim=1)  # (P,C,3)
    cluster_vectors = torch.stack(cluster_vectors, dim=1)  # (P,C,3)

    # select the cluster normal with the clostest cluster center
    distances = q.unsqueeze(-2) - cluster_centers
    distances = torch.linalg.vector_norm(distances, dim=-1)  # (Q, C)
    # reset the distances with no values
    idxs = torch.arange(distances.shape[1]).expand(distances.shape[0], -1).to(distances)
    mask = idxs > (cluster_idxs.max(dim=-1).values)[..., None]
    distances[mask] = torch.nan
    distances, indices = torch.topk(distances, 1, dim=1, largest=False)
    indices = indices[..., 0]  # just the top 1
    # the final cluster vectors
    vector = cluster_vectors[torch.arange(cluster_vectors.shape[0]), indices]

    # normalize the vector field to contain only normal vectors
    if normalize:
        vector /= torch.linalg.vector_norm(vector, dim=-1).unsqueeze(-1)
    vectors.append(vector)

vectors = torch.cat(vectors)

In [ ]:
points = data["points"]  # the surface points
normals = data["normals"]  # the normals on the surface
query = data["points"]  # the target points to evaluate the vector field

vectors = []
for q in torch.split(query, chunk_size):
    # compute the distances
    distances = torch.cdist(q, points, p=2)
    distances, indices = torch.topk(distances, 1, dim=1, largest=False)
    vector = normals[indices[..., 0]]
    # normalize the vector field to contain only normal vectors
    if normalize:
        vector /= torch.linalg.vector_norm(vector, dim=-1).unsqueeze(-1)
    vectors.append(vector)
v = torch.cat(vectors)


In [ ]:
normals

In [ ]:
v


In [ ]:
# compute the cluster vectors


# _cluster_centers = cluster_centers[:100]
# _query = query[:100]
# _cluster_vectors = cluster_vectors[:100]
# _cluster_idx = cluster_idxs[:100].max(dim=-1).values

# compute the distances
distances = q.unsqueeze(-2) - cluster_centers
distances = torch.linalg.vector_norm(distances, dim=-1)  # (Q, C)
# reset the distances with no values
idxs = torch.arange(distances.shape[1]).expand(distances.shape[0], -1).to(distances)
mask = idxs > (cluster_idxs.max(dim=-1).values)[..., None]
distances[mask] = torch.nan
distances, indices = torch.topk(distances, 1, dim=1, largest=False)
indices = indices[..., 0]  # just the top 1
# the final cluster vectors
vectors = cluster_vectors[torch.arange(cluster_vectors.shape[0]), indices]

In [ ]:
cluster_idxs[0]

In [ ]:
cluster_centers[0]

In [ ]:
query.shape, points.shape, cluster_centers.shape

In [ ]:
distances.shape
idxs = torch.arange(distances.shape[1]).expand(distances.shape[0], -1).to(distances)
mask = idxs > (_cluster_idx)[..., None]
distances[mask] = distances.max() + 1
distances

In [ ]:
cluster_centers[0]
query[0]
idx = cluster_idxs[0].max(dim=-1).values
# distances[0]
v = torch.linalg.vector_norm(query[0]-cluster_centers[0], dim=-1)
v[idx+1:] = v.max() + 1
v

In [ ]:
# BUG
indices < _cluster_idxs  # the indices should not be able to be bigger then cluster_idxs

In [ ]:
indices.squeeze(-1) <= _cluster_idxs.max(dim=-1).values

In [ ]:
indices.unsqueeze(-1).shape

In [ ]:
_cluster_idxs.max(dim=-1).values

In [ ]:
torch.isnan(cluster_vectors[:, 0, :]).sum()

In [ ]:
torch.lin

q = query[:100]  # (Q, 3)
distances = torch.cdist(cluster_centers, query[:100], p=2)
# distances, indices = torch.topk(distances, 1, dim=0, largest=False)
# cluster_vector.shape
distances.shape

In [ ]:
distances.shape

In [ ]:
idx = 5000
print(cluster_idxs[idx])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[indices[idx]].detach().cpu().numpy())
pcd.normals = o3d.utility.Vector3dVector(normals[indices[idx]].detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
idx = 5000
cluster = 4
print(cluster_idxs[idx])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[indices[idx]][cluster_idxs[idx]==cluster].detach().cpu().numpy())
pcd.normals = o3d.utility.Vector3dVector(normals[indices[idx]][cluster_idxs[idx]==cluster].detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
points[indices[idx]][cluster_idxs[idx]==0]

In [ ]:
normals[indices].shape

In [ ]:
k = 1
prev_idxs = cluster_idxs[:, :k]
# default is just the next index
prev_max_cluster = prev_idxs.max(dim=-1).values
current_idxs = prev_max_cluster + 1
current_idxs

In [ ]:
from neural_poisson.data.shapenet import ShapeNetCoreDataset
from pathlib import Path

model_id = "1a04e3eab45ca15dd86060f189eb133"
# define the paths
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCoreTiny/02691156"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"
dataset = ShapeNetCoreDataset(
    path=shapenet_path,
    segments=10,
    resolution=0.002,
    empty_space_max_ratio=1.0,
)

In [ ]:
dataset.points_surface

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(dataset.points_surface.detach().cpu().numpy())

pcd.normals = o3d.utility.Vector3dVector(dataset.vectors_surface.detach().cpu().numpy())
o3d.visualization.draw_plotly([pcd])

In [ ]:
if -1 > 0:
    print("test")

In [ ]:
dataset.points_close.shape